In [1]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from datetime import datetime, timezone, timedelta
from pyspark.sql.functions import *
import json, requests

StatementMeta(, 0d5b8f58-f866-415c-bc60-5f76d06729dc, 3, Finished, Available)

In [2]:
def AuthenticateWithSP():
    key_vault = ""
    tenant = "mon tenant ID"
    client = "mon client ID"
    client_secret = mssparkutils.credentials.getSecret(key_vault , "clientsecret")    
 
    try:
        from azure.identity import ClientSecretCredential
    except Exception:
        !pip install azure.identity
        from azure.identity import ClientSecretCredential
 
    # Generates the access token for the Service Principal
    api = 'https://analysis.windows.net/powerbi/api/.default'
    auth = ClientSecretCredential(authority = 'https://login.microsoftonline.com/',
                              tenant_id = tenant,
                              client_id = client,
                              client_secret = client_secret)
    access_token = auth.get_token(api)
    access_token = access_token.token
    header = {'Authorization': f'Bearer {access_token}'}
    print('\nSuccessfully authenticated.')
    return header

In [3]:
base_url = 'https://api.powerbi.com/v1.0/myorg/'
header= AuthenticateWithSP()

StatementMeta(, 0d5b8f58-f866-415c-bc60-5f76d06729dc, 5, Finished, Available)

In [9]:
#########################################################################################
# Variable definition
#########################################################################################

startDate= datetime.utcnow() + timedelta(days=-23) 
startDate = startDate.replace(hour=0, minute=00,second=00)
startDate=startDate.strftime("%Y-%m-%dT%H:%M:%S.000Z")


EndtDate= datetime.utcnow() + timedelta(days=-23) 
EndtDate = EndtDate.replace(hour=23, minute=59,second=59)
EndtDate=EndtDate.strftime("%Y-%m-%dT%H:%M:%S.000Z")

StatementMeta(, 0d5b8f58-f866-415c-bc60-5f76d06729dc, 11, Finished, Available)

In [10]:
API_url = base_url+"admin/activityevents?startDateTime='"+startDate+"'&endDateTime='"+EndtDate+"'"
response = requests.get(API_url, headers=header)

if response.status_code == 429:
    print("sleep")
    time.sleep(3601)
    header= AuthenticateWithSP()
    response = requests.get(API_url, headers=header)

JSONContent = json.loads(response.content)
JSONContentExplode = JSONContent['activityEventEntities']
lastResultSet = JSONContent['lastResultSet']

while lastResultSet==False :
    continuationUri = JSONContent['continuationUri']
    response = requests.get(continuationUri, headers=header)
    if response.status_code == 429:
        print("sleep")
        time.sleep(3601)
        header= AuthenticateWithSP()
        response = requests.get(API_url, headers=header)

    JSONContent = json.loads(response.content)
    JSONContentExplode=JSONContentExplode+JSONContent['activityEventEntities']
    lastResultSet = JSONContent['lastResultSet']


In [ ]:
if len(JSONContentExplode)>0:
    DailyActivityLogs=spark.read.json(sc.parallelize([json.dumps(JSONContentExplode)]))
    DailyActivityLogs= DailyActivityLogs.withColumn("CreationTime",to_timestamp(DailyActivityLogs.CreationTime, "yyyy-MM-dd'T'HH:mm:ss") )
    DailyActivityLogs=DailyActivityLogs.withColumn("PartitionDate",to_date( col("CreationTime")))

StatementMeta(, d562fad7-35b1-4e0f-9ad1-47ea79233ac9, 15, Finished, Available)

In [ ]:
spark.conf.set( "spark.sql.sources.partitionOverwriteMode", "dynamic")

if len(JSONContentExplode)>0:
    DailyActivityLogs.write.format('delta')\
    .mode("overwrite")\
    .option("overwriteSchema", "True")\
    .partitionBy("PartitionDate")\
    .save("Files/EventLogs")

StatementMeta(, d562fad7-35b1-4e0f-9ad1-47ea79233ac9, 16, Finished, Available)